## EDA Overview - Home-Credit Risk

In [1]:
import pandas as pd
import numpy as np

import matplotlib as plt

In [5]:
app_train = pd.read_csv('../data/raw/home_credit/application_train.csv')
app_test = pd.read_csv('../data/raw/home_credit/application_test.csv')

print(app_train.shape, app_test.shape)

(307511, 122) (48744, 121)
